# Multivariate Regression

import numpy as np
import statsmodels.api as sm

y = [1956.947,1884.65,1898.047,1898.811,1886.877]
#1956.947
#x = [[0,0,0,0,0,0,0],
#     [51, 55, 76, 45, 22, 22, 99],
#     [8, 81, 48, 80, 18, 31, 28],
 #    [80, 38, 3, 19, 52, 90, 9],
 #    [69, 71, 68, 13, 13, 29, 88]]

x = [[0,51,8,80,69],
     [0,55,81,38,71],
     [0,76,48,3,68],
     [0,45,80,19,13],
     [0,22,18,52,13],
     [0,22,31,90,29],
     [0,99,28,9,88]]
[0, 51, 51, 55, 76, 45, 22, 22, 99, 8, 8, 81, 48, 80, 18, 31, 28, 80, 80, 38, 3, 19, 52, 90, 9, 69, 69, 71, 68, 13, 13, 29, 88]

def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, X).fit()
    return results
reg_m(y,x)
print reg_m(y, x).summary()

In [2]:
# IMPORTANT:  RUN THIS BLOCK FIRST
# ONLY for loop tests for which all subcats have the same number of a single LID:
%matplotlib inline
from pymongo import MongoClient
from datetime import datetime
import matplotlib.pyplot as plt
def volume_reduction_and_runoff_volume_vs_lid_number(collectionName):
    mGal = 133680.5  # 1 Million gallon in cubic feet
    runs = db[collectionName]
    cursor = runs.find()
    numLists = []
    for n in range(0,7):
        numLists.append([])

    volReductionList = []
    totalSurfaceOutflowList = []
    csoVolList=[]
    noLidRun = cursor[0]
    noLidVolume = noLidRun['volume']
    outflow_series_list = []
    for run in cursor:
        for i in range(0,7):
            runParamsZero = run['runParamList'][i] 
            numInitial = runParamsZero['Number']
            numLists[i].append(numInitial)
        volume = run['volume']
        volReduction = noLidVolume-volume
        volReductionList.append(volReduction)
        outflow_series=run['outflow_series']
        #outflow_series_list.append
        series = run['outflow_series']
        #print series
        outflow_series_list.append(series)
        if run['lidDict'] == None:
            #numList.append(number)
            totalSurfaceOutflowList.append(None)
            continue
        else:
            totalSurfaceOutflow = 0.0
            runParamList = run['runParamList']
            lidDict = run['lidDict']
            for lidUsage in runParamList: #was lidUsage in runParamList 
                lid = lidUsage['LID']
                subcat = lidUsage['Subcat']
                lidNumber = lidUsage['Number']
                lidArea = lidUsage['Area']  # in square feet
                lidKey = subcat+' '+lid
                surfaceOutflowInches = lidDict[lidKey]['Surface Outflow']
                surfaceOutflow = surfaceOutflowInches*lidNumber*lidArea/12.0  # cubic feet
                totalSurfaceOutflow += surfaceOutflow/mGal  # convert to million Gal/year
            totalSurfaceOutflowList.append(totalSurfaceOutflow)

    return{"numLists": numLists, "outflow_series": outflow_series_list, "volReductionList": volReductionList, \
           "surface_outflow":totalSurfaceOutflowList,}

# Show the collections that have been generated

In [3]:
from pymongo import MongoClient

#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
collection = 'random_multivariate2'
client = MongoClient(mongoServer)
################################################

db = client[database]
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collectionNames = sorted(db.collection_names())
collectionNames

[u'2008',
 u'2008trial1',
 u'2008trial3',
 u'2008trial4',
 u'2008trial6',
 u'first_wingo_test',
 u'fourth_wingo_test',
 u'impervFixed',
 u'impervNotFixed',
 u'july',
 u'july10',
 u'july12',
 u'july13',
 u'july14',
 u'july15',
 u'july16',
 u'july17',
 u'july2',
 u'july21',
 u'july22',
 u'july23',
 u'july24',
 u'july3',
 u'july4',
 u'july5',
 u'july6',
 u'july7',
 u'july8',
 u'july9',
 u'largeTrialLinier',
 u'randomTrial1',
 u'randomTrial2',
 u'randomTrial4',
 u'randomTrial5',
 u'random_multivariate',
 u'random_multivariate2',
 u'second_wingo_test',
 u'trialWith%Perv',
 u'trialWith%Perv2',
 u'y16m10d11_demonstrate_runswmm_on_aws-2']

# use this function to calculate cso values 

In [4]:
def calculate_cso(outflow_values, ratio):
    cso_flow = 0
    hours = 0
    tot_flow = 0
    max_treatment = 3122*ratio #max cfs allowed to treatment
    #print max_treatment
    tot = len(outflow_values)
    for i in outflow_values: #out_variables is list within list (though outer list is just one element) (cfs/impervious acres)
        tot_flow += i
        if i > max_treatment:  #ratio method--- 
            cso = i - max_treatment # subtracting treated from total outflow
            cso_flow += cso 
            hours += 1
    tot_volume = tot_flow*900*7.48052 #convert to gallons, 900 seconds in 15 minutes
    cso_volume = cso_flow*900*7.48052 #for seconds in 15 minutes
    #equiv_rat = cso_volume/tot_volume  #equivalency ratio
    treated_volume = tot_volume - cso_volume
    #print tot_volume
    return {"cso_volume":cso_volume, "tot_volume":tot_volume}

#  Generates list of CSO outflow for each simulation

In [12]:
collectionName = 'random_multivariate2'
results = volume_reduction_and_runoff_volume_vs_lid_number(collectionName)
numSims = 5
# generates plot of cso reduction
cso_list = []
tot_vol_list = []
cso_reduction_list = []

for i in range(0,numSims):
    cso = calculate_cso(results["outflow_series"][i],0.05)
    cso_list.append(cso["cso_volume"])
    tot_vol_list.append(cso["tot_volume"])
    #reduction = cso["tot_volume"] - cso["cso_volume"]
    #reduction_list.append(reduction)
for i in range(0,numSims):
    original_cso_volume = cso_list[0]  #cso from run with no lids
    cso_reduction = original_cso_volume - cso_list[i]  
    cso_reduction_list.append(cso_reduction)
print cso_reduction_list

[0.0, 43717287.690467596, 35631014.630218506, 35974828.30604339, 42398989.6664511]


# Formats parameters to be used for multivariate regression

In [13]:
collectionName = 'random_multivariate2'
y = cso_reduction_list
results = volume_reduction_and_runoff_volume_vs_lid_number(collectionName)
x = results ['numLists']
print y
print x
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, X).fit()
    return results
reg_m(y,x)
print reg_m(y, x).summary()

[0.0, 43717287.690467596, 35631014.630218506, 35974828.30604339, 42398989.6664511]
[[0, 51, 8, 80, 69], [0, 55, 81, 38, 71], [0, 76, 48, 3, 68], [0, 45, 80, 19, 13], [0, 22, 18, 52, 13], [0, 22, 31, 90, 29], [0, 99, 28, 9, 88]]
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                     0.000
Date:                Sun, 04 Dec 2016   Prob (F-statistic):                nan
Time:                        14:32:54   Log-Likelihood:                 84.043
No. Observations:                   5   AIC:                            -158.1
Df Residuals:                       0   BIC:                            -160.0
Df Model:                           4                                         
Covariance Type:            nonrobust                        